---
format: 
  html:
    code-fold: true
    code-tools: true
    toc: true
    page-layout: full
execute:
  echo: true    # Shows code
  eval: true    # Runs code
  output: false # Should hide standard outputs
  warning: false
  error: false
  message: false
---

# Network Service Gap Methods & Analysis

In [1]:
#| output: false
import os
import json
import pickle
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import osmnx as ox
import geopy.distance
import requests 
from scipy import stats
from tqdm import tqdm
from shapely.geometry import Point, LineString, MultiPoint
from shapely.ops import unary_union
from shapely.geometry import Polygon
from dotenv import load_dotenv
import random
from datetime import datetime
import folium
from shapely.geometry import mapping as shapely_mapping

# Print current working directory
print(f"Working directory: {os.getcwd()}")

# Set OSMnx cache location explicitly
ox.settings.cache_folder = os.path.join('data', 'cache', 'osmnx')
print(f"OSMnx cache folder: {ox.settings.cache_folder}")

# For our cache
CACHE_DIR = os.path.join('data', 'cache')
os.makedirs(CACHE_DIR, exist_ok=True)

# Set global debug mode (set to True for detailed component outputs)
DEBUG_MODE = True

# Define global thresholds for gap scoring - used in multiple functions
THRESHOLDS = [0.45, 0.55, 0.65]  # Adjusted thresholds for better distribution

# Using NetworkX with point snapping

print("Using NetworkX with point snapping for network analysis.")

def load_data():
    """Load census, boundary, and EV station data."""
    load_dotenv()
    api_key = os.getenv('OPENCHARGE_API_KEY')
    if not api_key:
        raise EnvironmentError('OPENCHARGE_API_KEY is not set.')

    # Census data
    census = gpd.read_file('data/phila_census1.gpkg')
    census = census[census['total_pop'] > 0]
    census = census[census['pop_density'] >= 1000]
    census = census.to_crs('EPSG:4326')

    # City boundary
    boundary = gpd.read_file('data/City_Limits.geojson').to_crs('EPSG:4326')

    # EV stations
    params = {
        'key': api_key,
        'countrycode': 'US',
        'maxresults': 1000,
        'latitude': 39.9526,
        'longitude': -75.1652,
        'distance': 10,
        'distanceunit': 'km',
        'compact': True,
        'verbose': False,
        'output': 'json'
    }
    resp = requests.get('https://api.openchargemap.io/v3/poi', params=params, timeout=10)
    resp.raise_for_status()
    data = resp.json()

    records = []
    for s in data:
        lat = s.get('AddressInfo',{}).get('Latitude')
        lon = s.get('AddressInfo',{}).get('Longitude')
        if lat is None or lon is None:
            continue
        pt = Point(lon, lat)
        if not pt.within(boundary.geometry.iloc[0]):
            continue
        conns = s.get('Connections', [])
        max_kw = max((c.get('PowerKW',0) for c in conns), default=0)
        records.append({
            'id': s.get('ID'),
            'name': s.get('AddressInfo',{}).get('Title',''),
            'num_points': len(conns),
            'max_power': max_kw,
            'geometry': pt
        })
    stations = gpd.GeoDataFrame(records, crs='EPSG:4326')

    # Project to state plane for spatial operations
    census = census.to_crs('EPSG:2272')
    boundary = boundary.to_crs('EPSG:2272')
    stations = stations.to_crs('EPSG:2272')

    # Add charging speed categories
    stations['charging_speed'] = pd.cut(
        stations['max_power'],
        bins=[-np.inf, 7, 50, np.inf],
        labels=['Level 1 (Slow)', 'Level 2 (Medium)', 'DC Fast (Rapid)']
    )

    # Validation tracking
    validation = {
        'total_stations_from_api': len(data),
        'stations_within_boundary': len(records),
        'excluded_stations': len(data) - len(records),
        'exclusion_reasons': {
            'outside_boundary': sum(1 for s in data if s.get('AddressInfo',{}).get('Latitude') and
                                   s.get('AddressInfo',{}).get('Longitude') and
                                   not Point(s.get('AddressInfo',{}).get('Longitude'), 
                                            s.get('AddressInfo',{}).get('Latitude')).within(boundary.geometry.iloc[0])),
            'missing_coordinates': sum(1 for s in data if not s.get('AddressInfo',{}).get('Latitude') or
                                      not s.get('AddressInfo',{}).get('Longitude'))
        }
    }

    return census, boundary, stations, validation


def create_networks():
    """Load or build walk and drive networks and cache them."""
    cache = 'data/network_cache.pkl'
    if os.path.exists(cache):
        with open(cache,'rb') as f:
            return pickle.load(f)

    G_walk = ox.graph_from_place('Philadelphia, Pennsylvania', network_type='walk', simplify=False)
    G_walk = ox.project_graph(G_walk, to_crs='EPSG:2272')
    G_walk = ox.distance.add_edge_lengths(G_walk)

    G_drive = ox.graph_from_place('Philadelphia, Pennsylvania', network_type='drive', simplify=False)
    G_drive = ox.project_graph(G_drive, to_crs='EPSG:2272')
    G_drive = ox.distance.add_edge_lengths(G_drive)

    os.makedirs('data', exist_ok=True)
    with open(cache,'wb') as f:
        pickle.dump((G_walk, G_drive),f)

    def check_network_connectivity(network, network_type):
        """Check and report on network connectivity issues."""
        # Find connected components
        connected_components = list(nx.connected_components(network.to_undirected()))
        print(f"{network_type} network has {len(connected_components)} connected components")
        
        # Report on largest component
        largest_component = max(connected_components, key=len)
        largest_pct = len(largest_component) / network.number_of_nodes() * 100
        print(f"Largest component contains {largest_pct:.1f}% of all nodes")
        
        if len(connected_components) > 1:
            print("WARNING: Network has disconnected components, which may cause routing failures")
        
        return len(connected_components)

    # Add after creating networks:
    print("Checking network connectivity...")
    walk_components = check_network_connectivity(G_walk, "Walking")
    drive_components = check_network_connectivity(G_drive, "Driving")

    return G_walk, G_drive


def find_nearest_node(network, point_x, point_y, fallback_tolerance=5000):
    """
    Find the nearest node to a point using robust search methods.
    Tries OSMnx nearest_nodes first, then falls back to manual distance calculation
    with an increasing search radius if needed.
    
    Parameters:
    -----------
    network : networkx.Graph
        Road network from OSMnx
    point_x, point_y : float
        Coordinates of the target point
    fallback_tolerance : float
        Maximum distance to search for nodes if nearest_nodes fails
        
    Returns:
    --------
    int or None
        Node ID of nearest node, or None if no node found within tolerance
    """
    # First try with OSMnx's built-in function
    try:
        return ox.nearest_nodes(network, point_x, point_y)
    except Exception as e:
        if DEBUG_MODE:
            print(f"Standard nearest_nodes failed: {e}, trying manual search...")
    
    # Manual fallback with increasing radius
    try:
        # Get all nodes and their coordinates
        nodes = list(network.nodes)
        if not nodes:
            if DEBUG_MODE:
                print("No nodes in network!")
            return None
            
        # Use vectorized operations for efficiency
        node_coords = np.array([
            [network.nodes[n].get('x', 0), network.nodes[n].get('y', 0)] 
            for n in nodes
        ])
        
        point_coords = np.array([point_x, point_y])
        
        # Calculate squared distances to all nodes
        squared_distances = np.sum((node_coords - point_coords)**2, axis=1)
        
        # Get the node with minimum distance
        min_idx = np.argmin(squared_distances)
        min_distance = np.sqrt(squared_distances[min_idx])
        
        if min_distance <= fallback_tolerance:
            if DEBUG_MODE and min_distance > 100:
                print(f"Found node at distance {min_distance:.1f} units")
            return nodes[min_idx]
        else:
            if DEBUG_MODE:
                print(f"Nearest node is too far: {min_distance:.1f} units > {fallback_tolerance}")
            return None
            
    except Exception as e:
        if DEBUG_MODE:
            print(f"Manual nearest node search failed: {e}")
        return None


def map_match_point(network, point_x, point_y, tolerance=5000):
    """
    Map match a point to the nearest edge in the network and return
    the nearest node along that edge.
    
    This provides better connectivity than just finding the nearest node,
    especially when points are far from actual network nodes.
    
    Parameters:
    -----------
    network : networkx.Graph
        Road network from OSMnx
    point_x, point_y : float
        Coordinates of the target point
    tolerance : float
        Maximum distance to search
        
    Returns:
    --------
    int or None
        Node ID of matched node, or None if no match found
    """
    try:
        # Create a Point object for the input coordinates
        point = Point(point_x, point_y)
        
        # Find nearest edges, not just nodes
        best_edge = None
        best_dist = float('inf')
        
        # Check a sample of edges for efficiency (every 10th edge)
        edges_sample = list(network.edges(data=True))[::10]
        
        for u, v, data in edges_sample:
            if 'geometry' in data:
                # If edge has a geometry attribute, use it
                edge_geom = data['geometry']
                dist = edge_geom.distance(point)
            else:
                # Otherwise, create a line between nodes
                try:
                    u_x, u_y = network.nodes[u]['x'], network.nodes[u]['y']
                    v_x, v_y = network.nodes[v]['x'], network.nodes[v]['y']
                    line = [(u_x, u_y), (v_x, v_y)]
                    edge_geom = LineString(line)
                    dist = edge_geom.distance(point)
                except KeyError:
                    # Skip if nodes don't have coordinates
                    continue
            
            # Update best match
            if dist < best_dist:
                best_dist = dist
                best_edge = (u, v)
        
        # Return the closer node from the best edge
        if best_edge and best_dist <= tolerance:
            u, v = best_edge
            u_dist = ((network.nodes[u]['x'] - point_x)**2 + 
                      (network.nodes[u]['y'] - point_y)**2)**0.5
            v_dist = ((network.nodes[v]['x'] - point_x)**2 + 
                      (network.nodes[v]['y'] - point_y)**2)**0.5
            return u if u_dist < v_dist else v
        
        # Fallback to regular nearest node if no good edge found
        return find_nearest_node(network, point_x, point_y, fallback_tolerance=tolerance)
    
    except Exception as e:
        if DEBUG_MODE:
            print(f"Map matching failed: {e}")
        # Fallback to regular nearest node
        return find_nearest_node(network, point_x, point_y, fallback_tolerance=tolerance)


def enhance_network_connectivity(G):
    """
    Enhance network connectivity by ensuring key locations are connected.
    
    Parameters:
    -----------
    G : networkx.Graph
        Road network to enhance
    
    Returns:
    --------
    networkx.Graph
        Enhanced network
    """
    # Add self-loops to all nodes to ensure they can be reached from themselves
    # This helps with isolated nodes
    for node in G.nodes():
        if not G.has_edge(node, node):
            G.add_edge(node, node, length=0)
    
    # Check the number of connected components
    connected_components = list(nx.connected_components(G.to_undirected()))
    largest_component = max(connected_components, key=len)
    
    # If we have multiple components, add edges to connect them to largest component
    if len(connected_components) > 1:
        largest_component_nodes = list(largest_component)
        for component in connected_components:
            if component != largest_component:
                # Find the closest node pair between components
                min_dist = float('inf')
                best_pair = None
                
                # Sample nodes from each component for efficiency
                component_nodes = list(component)
                sample_size = min(10, len(component_nodes))
                sampled_component = random.sample(component_nodes, sample_size)
                
                sample_size_largest = min(20, len(largest_component_nodes))
                sampled_largest = random.sample(largest_component_nodes, sample_size_largest)
                
                # Find the closest pair
                for n1 in sampled_component:
                    for n2 in sampled_largest:
                        try:
                            x1, y1 = G.nodes[n1]['x'], G.nodes[n1]['y']
                            x2, y2 = G.nodes[n2]['x'], G.nodes[n2]['y']
                            dist = ((x1-x2)**2 + (y1-y2)**2)**0.5
                            if dist < min_dist:
                                min_dist = dist
                                best_pair = (n1, n2)
                        except KeyError:
                            continue
                
                # Add an edge between the closest nodes
                if best_pair:
                    G.add_edge(best_pair[0], best_pair[1], length=min_dist)
    
    return G


def calculate_multimodal_distance_batch(origins, destinations, walk_net, drive_net):
    """
    Calculate minimum walking and driving distances from origins to destinations using NetworkX.
    
    Parameters
    ----------
    origins : dict
        Dictionary mapping IDs to Point geometries for origins
    destinations : list
        List of Point geometries for destinations
    walk_net : networkx.Graph
        Walking network 
    drive_net : networkx.Graph
        Driving network
        
    Returns
    -------
    tuple
        (walk_distances, drive_distances) dictionaries mapping (origin_id, destination_idx)
        to distances in meters
    """
    # Check if we should use cached distances
    cache_file = os.path.join(CACHE_DIR, 'distance_cache.pkl')
    
    # Create cache dir if needed
    os.makedirs(CACHE_DIR, exist_ok=True)
    
    # Try to load from cache
    if os.path.exists(cache_file):
        try:
            print("Loading distances from cache...")
            with open(cache_file, 'rb') as f:
                distances = pickle.load(f)
            walk_d, drive_d = distances
            print(f"Loaded {len(walk_d)} walking and {len(drive_d)} driving distances from cache.")
            return walk_d, drive_d
        except Exception as e:
            print(f"Error loading distance cache: {e}")
            # Continue with calculation
    
    # Calculate distances with NetworkX
    print("Calculating distances with NetworkX...")
    walk_distances, drive_distances = calculate_distances_with_networkx(
        origins, destinations, walk_net, drive_net
    )
    
    # Cache the results
    try:
        with open(cache_file, 'wb') as f:
            pickle.dump((walk_distances, drive_distances), f)
        print(f"Saved distance calculations to {cache_file}")
    except Exception as e:
        print(f"Failed to save distance cache: {e}")
    
    return walk_distances, drive_distances


def calculate_distances_with_networkx(origins, destinations, walk_net, drive_net):
    """
    Calculate distances between origins and destinations using NetworkX.
    
    Parameters
    ----------
    origins : dict
        Dictionary mapping IDs to Point geometries for origins
    destinations : list
        List of Point geometries for destinations
    walk_net : networkx.Graph
        Walking network 
    drive_net : networkx.Graph
        Driving network
        
    Returns
    -------
    tuple
        (walk_distances, drive_distances) dictionaries mapping (origin_id, destination_idx)
        to distances in meters
    """
    # Prepare outputs
    walk_distances = {}
    drive_distances = {}
    
    # Track failures for reporting
    walk_failures = 0
    drive_failures = 0
    total_pairs = len(origins) * len(destinations)
    
    # Define distance filter threshold in meters
    # This is based on the maximum distance band used in the methodology (15,840 ft ≈ 4.8 km)
    # Adding a small buffer to ensure we don't miss any relevant connections
    MAX_DISTANCE_FILTER = 6000  # 6 km
    
    # Track stats for filtered pairs
    skipped_pairs = 0
    processed_pairs = 0
    
    print(f"Using distance filter of {MAX_DISTANCE_FILTER/1000:.1f} km")
    
    # Process each origin-destination pair
    for origin_id, origin_geom in tqdm(origins.items(), desc="Calculating distances"):
        origin_x, origin_y = origin_geom.x, origin_geom.y
        
        # Find nearest nodes once per origin to speed things up
        try:
            origin_node_walk = ox.distance.nearest_nodes(walk_net, origin_x, origin_y)
            origin_node_drive = ox.distance.nearest_nodes(drive_net, origin_x, origin_y)
        except Exception as e:
            if DEBUG_MODE:
                print(f"Failed to find nearest nodes for origin {origin_id}: {str(e)[:100]}...")
            # Skip this origin if we can't find nodes
            continue
        
        # Pre-filter destinations based on Euclidean distance
        filtered_destinations = []
        for dest_idx, dest_geom in enumerate(destinations):
            # Calculate straight-line distance
            dest_x, dest_y = dest_geom.x, dest_geom.y
            straight_dist = ((origin_x - dest_x)**2 + (origin_y - dest_y)**2)**0.5
            
            # Only process destinations within the threshold distance
            if straight_dist <= MAX_DISTANCE_FILTER:
                filtered_destinations.append((dest_idx, dest_geom))
            else:
                # For points beyond our filter, set to infinity (they'll get the maximum score anyway)
                walk_distances[(origin_id, dest_idx)] = float('inf')
                drive_distances[(origin_id, dest_idx)] = float('inf')
                skipped_pairs += 1
        
        # Process each filtered destination for this origin
        for dest_idx, dest_geom in filtered_destinations:
            processed_pairs += 1
            dest_x, dest_y = dest_geom.x, dest_geom.y
            
            # Try walking distance calculation
            try:
                dest_node = ox.distance.nearest_nodes(walk_net, dest_x, dest_y)
                path_length = nx.shortest_path_length(
                    walk_net, origin_node_walk, dest_node, weight='length'
                )
                walk_distances[(origin_id, dest_idx)] = path_length
            except Exception as e:
                # Fall back to straight-line distance with 1.3 detour factor for walking
                # This is a reasonable approximation for urban areas
                if DEBUG_MODE:
                    print(f"Walking path failed for ({origin_id}, {dest_idx}): {str(e)[:100]}...")
                
                straight_dist = geopy.distance.geodesic(
                    (origin_y, origin_x), (dest_y, dest_x)
                ).meters
                
                # Apply detour factor - walking routes tend to be ~1.3x straight line distance
                walk_distances[(origin_id, dest_idx)] = straight_dist * 1.3
                walk_failures += 1
            
            # Try driving distance calculation
            try:
                dest_node = ox.distance.nearest_nodes(drive_net, dest_x, dest_y)
                path_length = nx.shortest_path_length(
                    drive_net, origin_node_drive, dest_node, weight='length'
                )
                drive_distances[(origin_id, dest_idx)] = path_length
            except Exception as e:
                # Fall back to straight-line distance with 1.5 detour factor for driving
                # This is a reasonable approximation for urban areas
                if DEBUG_MODE:
                    print(f"Driving path failed for ({origin_id}, {dest_idx}): {str(e)[:100]}...")
                
                straight_dist = geopy.distance.geodesic(
                    (origin_y, origin_x), (dest_y, dest_x)
                ).meters
                
                # Apply detour factor - driving routes tend to be ~1.5x straight line distance
                drive_distances[(origin_id, dest_idx)] = straight_dist * 1.5
                drive_failures += 1
    
    # Report on statistics
    filter_percent = skipped_pairs / total_pairs * 100 if total_pairs > 0 else 0
    print(f"Distance filter: Processed {processed_pairs} pairs, skipped {skipped_pairs} pairs ({filter_percent:.1f}% reduction)")
    
    # Report on failures
    if walk_failures > 0:
        print(f"Warning: {walk_failures}/{processed_pairs} walking distance calculations failed ({walk_failures/processed_pairs*100:.1f}%)")
        print("Used straight-line distance with detour factor as fallback.")
    if drive_failures > 0:
        print(f"Warning: {drive_failures}/{processed_pairs} driving distance calculations failed ({drive_failures/processed_pairs*100:.1f}%)")
        print("Used straight-line distance with detour factor as fallback.")
    
    return walk_distances, drive_distances


def calculate_coverage_scores(census, walk_dist, drive_dist, stations):
    """Calculate coverage scores with station capacity and power factored in."""
    # Create station quality index (combine points count and power)
    stations['quality_index'] = stations['num_points'] * np.sqrt(stations['max_power'] / 50)
    # Normalize quality to 0.5-1.5 range (0.5=worst, 1.0=average, 1.5=best)
    min_q = stations['quality_index'].min()
    max_q = stations['quality_index'].max()
    stations['quality_factor'] = 0.5 + ((stations['quality_index'] - min_q) / (max_q - min_q)) if max_q > min_q else 1.0
    
    # Build station lookup by geometry for quick access
    station_lookup = {geom.wkt: factor for geom, factor in zip(stations.geometry, stations['quality_factor'])}
    
    results = []
    for idx, row in census.iterrows():
        w = walk_dist.get(idx, np.inf)
        d = drive_dist.get(idx, np.inf)
        
        # Find nearest station and its quality factor
        nearest_dist = min(w, d)
        nearest_station_geom = min(
            [pt for pt in stations.geometry],
            key=lambda pt: ((pt.x - row.geometry.centroid.x)**2 + (pt.y - row.geometry.centroid.y)**2)**0.5
        )
        quality_factor = station_lookup.get(nearest_station_geom.wkt, 1.0)
        
        # Calculate base coverage as before
        ws = 0.7 if w<=1320 else 0.5 if w<=2640 else 0.3 if w<=3960 else 1.0
        ds = 0.7 if d<=5280 else 0.5 if d<=10560 else 0.3 if d<=15840 else 1.0
        combined = 0.4*ws + 0.6*ds
        
        # Adjust coverage by station quality
        adjusted_combined = combined * quality_factor
        
        # Apply density adjustment
        dens = row['pop_density']/census['pop_density'].max()
        score = adjusted_combined * (1 - 0.3*dens)
        
        results.append((score, w, d))
        
    df = pd.DataFrame(results, index=census.index, columns=['score_raw','walk_ft','drive_ft'])
    minv, maxv = df['score_raw'].min(), df['score_raw'].max()
    df['coverage_score'] = (df['score_raw'] - minv)/(maxv-minv)
    return df


def build_service_areas(stations, walk_net, drive_net):
    """
    Build service area polygons for the stations using NetworkX.
    
    Parameters
    ----------
    stations : GeoDataFrame
        GeoDataFrame of EV charging stations
    walk_net : networkx.Graph
        Walking network
    drive_net : networkx.Graph
        Driving network
        
    Returns
    -------
    dict
        Dictionary of service area polygons by type and distance
    """
    # Check if we should use cached service areas
    cache_file = os.path.join(CACHE_DIR, 'service_areas_cache.pkl')
    
    # Try to load from cache
    if os.path.exists(cache_file):
        try:
            print("Loading service areas from cache...")
            with open(cache_file, 'rb') as f:
                service_areas = pickle.load(f)
            print(f"Loaded {len(service_areas)} service areas from cache")
            return service_areas
        except Exception as e:
            print(f"Error loading service areas cache: {e}")
            # Continue with calculation
    
    # Calculate service areas if not cached
    print("Building service areas using NetworkX...")
    service_areas = build_service_areas_with_networkx(stations, walk_net, drive_net)
    
    # Cache the results
    try:
        with open(cache_file, 'wb') as f:
            pickle.dump(service_areas, f)
        print(f"Saved service areas to {cache_file}")
    except Exception as e:
        print(f"Failed to save service areas cache: {e}")
    
    return service_areas


def build_service_areas_with_networkx(stations, walk_net, drive_net):
    """
    Legacy method to build service areas using NetworkX.
    
    Parameters
    ----------
    stations : GeoDataFrame
        GeoDataFrame of EV charging stations
    walk_net : networkx.Graph
        Walking network
    drive_net : networkx.Graph
        Driving network
        
    Returns
    -------
    dict
        Dictionary of service area polygons by type and distance
    """
    # Initialize result dictionary
    service_areas = {}
    
    # Define service area distances
    walk_distances = [400, 800]  # meters (approx. 5 and 10 min walk)
    drive_distances = [1000, 3000, 8000]  # meters
    
    # Track successes and failures
    success_count = 0
    error_count = 0
    
    # Create walking service areas
    for dist in walk_distances:
        print(f"Creating walking service area for {dist}m...")
        area_key = f'walk_{dist}m'
        reachable_nodes = []
        
        # For each station, find reachable nodes
        for idx, station in stations.iterrows():
            try:
                # Find nearest node to station
                start_node = ox.distance.nearest_nodes(walk_net, station.geometry.x, station.geometry.y)
                
                # Get reachable nodes within distance
                reachable = nx.single_source_dijkstra_path_length(
                    walk_net, start_node, cutoff=dist, weight='length'
                )
                
                # Get coordinates of reachable nodes
                for node_id in reachable:
                    x, y = walk_net.nodes[node_id]['x'], walk_net.nodes[node_id]['y']
                    reachable_nodes.append(Point(x, y))
            except Exception as e:
                if DEBUG_MODE:
                    print(f"Error calculating walking service area for station {idx}: {str(e)[:100]}...")
                continue
        
        # Create service area from reachable nodes
        if len(reachable_nodes) >= 3:
            try:
                # Create convex hull from reachable nodes
                multi_point = MultiPoint(reachable_nodes)
                service_areas[area_key] = multi_point.convex_hull
                success_count += 1
            except Exception as e:
                if DEBUG_MODE:
                    print(f"Error creating convex hull for {area_key}: {str(e)[:100]}...")
                # Fallback to buffer
                service_areas[area_key] = stations.geometry.unary_union.buffer(dist)
                error_count += 1
        else:
            # If too few reachable nodes, use buffer
            print(f"Too few reachable nodes ({len(reachable_nodes)}) for {area_key}. Using buffer.")
            service_areas[area_key] = stations.geometry.unary_union.buffer(dist)
            error_count += 1
    
    # Create driving service areas
    for dist in drive_distances:
        print(f"Creating driving service area for {dist}m...")
        area_key = f'drive_{dist}m'
        reachable_nodes = []
        
        # For each station, find reachable nodes
        for idx, station in stations.iterrows():
            try:
                # Find nearest node to station
                start_node = ox.distance.nearest_nodes(drive_net, station.geometry.x, station.geometry.y)
                
                # Get reachable nodes within distance
                reachable = nx.single_source_dijkstra_path_length(
                    drive_net, start_node, cutoff=dist, weight='length'
                )
                
                # Get coordinates of reachable nodes
                for node_id in reachable:
                    x, y = drive_net.nodes[node_id]['x'], drive_net.nodes[node_id]['y']
                    reachable_nodes.append(Point(x, y))
            except Exception as e:
                if DEBUG_MODE:
                    print(f"Error calculating driving service area for station {idx}: {str(e)[:100]}...")
                continue
        
        # Create service area from reachable nodes
        if len(reachable_nodes) >= 3:
            try:
                # Create convex hull from reachable nodes
                multi_point = MultiPoint(reachable_nodes)
                service_areas[area_key] = multi_point.convex_hull
                success_count += 1
            except Exception as e:
                if DEBUG_MODE:
                    print(f"Error creating convex hull for {area_key}: {str(e)[:100]}...")
                # Fallback to buffer
                service_areas[area_key] = stations.geometry.unary_union.buffer(dist)
                error_count += 1
        else:
            # If too few reachable nodes, use buffer
            print(f"Too few reachable nodes ({len(reachable_nodes)}) for {area_key}. Using buffer.")
            service_areas[area_key] = stations.geometry.unary_union.buffer(dist)
            error_count += 1
    
    print(f"Service area generation: {success_count} successes, {error_count} fallbacks to buffers")
    return service_areas


def perform_equity_analysis(census, areas):
    """Compute effect size and confidence intervals for equity metrics."""
    # Check if we should use cached equity results
    cache_file = os.path.join(CACHE_DIR, 'equity_analysis_cache.pkl')
    
    # Try to load from cache
    if os.path.exists(cache_file):
        try:
            print("Loading equity analysis from cache...")
            with open(cache_file, 'rb') as f:
                equity = pickle.load(f)
            print(f"Loaded equity analysis for {len(equity)} service areas from cache")
            return equity
        except Exception as e:
            print(f"Error loading equity analysis cache: {e}")
            # Continue with calculation
    
    # Perform equity analysis if not cached
    print("Computing equity analysis...")
    equity = {}
    for label, geom in areas.items():
        covered = census[census.geometry.intersects(geom)]
        not_covered = census[~census.index.isin(covered.index)]
        metrics = {}
        for var in ['median_income','poverty_rate','pop_density','bach_degree_rate']:
            a = covered[var].dropna()
            b = not_covered[var].dropna()
            if len(a)>1 and len(b)>1:
                pooled = np.sqrt(((len(a)-1)*a.var()+(len(b)-1)*b.var())/(len(a)+len(b)-2))
                d = (a.mean()-b.mean())/pooled
                se = pooled*np.sqrt(1/len(a)+1/len(b))
                ci = (d-1.96*se, d+1.96*se)
                pval = stats.ttest_ind(a,b).pvalue
                metrics[var] = {'effect_size':d,'ci_lower':ci[0],'ci_upper':ci[1],'p_value':pval}
        equity[label] = metrics
    
    # Cache the results
    try:
        with open(cache_file, 'wb') as f:
            pickle.dump(equity, f)
        print(f"Saved equity analysis to {cache_file}")
    except Exception as e:
        print(f"Failed to save equity analysis cache: {e}")
    
    return equity


# ----------------------------------------------------------
# Compute dynamic weights based on equity disparities
def calculate_dynamic_weights(equity_results):
    # Base weights for each component - starting point
    weights = {'coverage':0.30, 'income':0.20, 'poverty':0.20, 'density':0.15, 'education':0.15}
    
    # Track effect sizes and significance by variable across all buffer types
    effect_sizes = {'income': 0, 'poverty': 0, 'density': 0, 'education': 0}
    significant_effects = {'income': False, 'poverty': False, 'density': False, 'education': False}
    
    # Map variable names to weight keys
    var_to_weight = {
        'median_income': 'income',
        'poverty_rate': 'poverty',
        'pop_density': 'density',
        'bach_degree_rate': 'education'
    }
    
    # First pass: collect maximum effect sizes and check significance
    for buf, metrics in equity_results.items():
        for var, stats in metrics.items():
            if var in var_to_weight:
                weight_key = var_to_weight[var]
                # Track the highest absolute effect size found
                effect_sizes[weight_key] = max(effect_sizes[weight_key], abs(stats['effect_size']))
                # Mark as significant if p-value < 0.05
                if stats['p_value'] < 0.05:
                    significant_effects[weight_key] = True
    
    # Adjust weights based on effect sizes and significance
    for weight_key, effect_size in effect_sizes.items():
        is_significant = significant_effects[weight_key]
        
        if is_significant:
            # For significant effects, apply progressive scaling:
            # Small effect (0.2-0.5): +5%
            # Medium effect (0.5-0.8): +10%
            # Large effect (>0.8): +15%
            if effect_size > 0.8:
                weights[weight_key] += 0.15
            elif effect_size > 0.5:
                weights[weight_key] += 0.10
            elif effect_size > 0.2:
                weights[weight_key] += 0.05
    
    # Normalize weights to sum to 1
    total = sum(weights.values())
    normalized_weights = {k: v/total for k, v in weights.items()}
    
    if DEBUG_MODE:
        print("Original weights:", weights)
        print("Effect sizes:", effect_sizes)
        print("Significant effects:", significant_effects)
        print("Normalized weights:", normalized_weights)
    
    return normalized_weights


# ----------------------------------------------------------
# Apply final gap scoring by combining coverage, income, poverty, density, and education
def apply_gap_scoring(census, dyn_wts):
    """
    Apply a simpler gap scoring methodology using:
    1. Linear combinations of normalized components
    2. Quantile-based thresholds for natural data distribution
    """
    # Step 1: Create simple normalized components (0-1 scale)
    
    # Coverage component (higher coverage => lower gap)
    cov_norm = 1 - census['coverage_score']  # Invert so higher = worse coverage
    cov_comp = cov_norm * dyn_wts['coverage']
    
    # Income component (lower income => higher gap)
    inc_norm = (census['median_income'].max() - census['median_income']) / (census['median_income'].max() - census['median_income'].min())
    inc_comp = inc_norm * dyn_wts['income']
    
    # Poverty component (higher poverty => higher gap)
    pov_norm = (census['poverty_rate'] - census['poverty_rate'].min()) / (census['poverty_rate'].max() - census['poverty_rate'].min())
    pov_comp = pov_norm * dyn_wts['poverty']
    
    # Density component (higher pop density => higher gap)
    den_norm = (census['pop_density'] - census['pop_density'].min()) / (census['pop_density'].max() - census['pop_density'].min())
    den_comp = den_norm * dyn_wts['density']
    
    # Education component (lower education => higher gap)
    edu_norm = (census['bach_degree_rate'].max() - census['bach_degree_rate']) / (census['bach_degree_rate'].max() - census['bach_degree_rate'].min())
    edu_comp = edu_norm * dyn_wts['education']
    
    # Step 2: Linear combination of components (simple weighted sum)
    census['gap_score'] = cov_comp + inc_comp + pov_comp + den_comp + edu_comp
    
    # Step 3: Use fixed thresholds based on meaningful categories rather than quartiles
    # Low, Medium, High, Critical priority with more tracts in lower categories
    # These fixed thresholds create a right-skewed distribution
    # Using global THRESHOLDS instead of defining locally
    
    if DEBUG_MODE:
        print(f"Fixed thresholds: {THRESHOLDS}")
    
    labels = ['Low Priority','Medium Priority','High Priority','Critical Priority']
    census['gap_category'] = pd.cut(census['gap_score'], 
                                   bins=[-np.inf] + THRESHOLDS + [np.inf], 
                                   labels=labels, 
                                   include_lowest=True)

    # Print component statistics for debugging
    if DEBUG_MODE:
        print("\n=== GAP SCORE COMPONENTS ===")
        print(f"Coverage component (weight: {dyn_wts['coverage']:.2f}):")
        print(cov_comp.describe())
        print(f"\nIncome component (weight: {dyn_wts['income']:.2f}):")
        print(inc_comp.describe())
        print(f"\nPoverty component (weight: {dyn_wts['poverty']:.2f}):")
        print(pov_comp.describe())
        print(f"\nDensity component (weight: {dyn_wts['density']:.2f}):")
        print(den_comp.describe())
        print(f"\nEducation component (weight: {dyn_wts['education']:.2f}):")
        print(edu_comp.describe())

    # Print final gap score statistics
    if DEBUG_MODE:
        print("\n=== GAP SCORE STATISTICS ===")
        print(census['gap_score'].describe())
        
        # Print distribution of priority categories
        print("\n=== PRIORITY CATEGORY DISTRIBUTION ===")
        cat_counts = census['gap_category'].value_counts().sort_index()
        for category, count in cat_counts.items():
            percentage = (count / len(census)) * 100
            print(f"{category}: {count} tracts ({percentage:.1f}%)")

    return census


# ----------------------------------------------------------
# Compute coverage statistics by demographic category
def compute_coverage_by_demographics(census):
    demo_cov = {}
    # Loop through each demographic dimension
    for col in ['income_category','education_category','density_category']:
        demo_cov[col] = {}
        for cat in census[col].unique():
            sub = census[census[col] == cat]
            total_pop = int(sub['total_pop'].sum())
            covered_pop = int(sub[sub['is_served']]['total_pop'].sum())
            pct = covered_pop / total_pop * 100 if total_pop>0 else 0
            demo_cov[col][cat] = {'total_population': total_pop, 'covered_population': covered_pop, 'coverage_percent': pct}
    return demo_cov


# ----------------------------------------------------------
# Save all pipeline outputs: GeoPackage, JSON/CSV stats, and README
def save_pipeline_outputs(census, stations, areas, equity_res, demo_cov, dyn_wts):
    os.makedirs('data', exist_ok=True)
    # Save census tracts and stations
    census.to_file('data/gap_analysis_fixed.gpkg', layer='census', driver='GPKG')
    stations.to_file('data/gap_analysis_fixed.gpkg', layer='stations', driver='GPKG')
    # Save each service area as its own layer
    for label, geom in areas.items():
        gdf = gpd.GeoDataFrame(geometry=[geom], crs=census.crs)
        gdf.to_file('data/gap_analysis_fixed.gpkg', layer=f'service_area_{label}', driver='GPKG')
    # Equity analysis JSON & CSV
    with open('data/equity_analysis.json','w') as f: json.dump(equity_res, f, indent=2)
    eq_list = []
    for buf, mets in equity_res.items():
        for var, st in mets.items(): eq_list.append({'buffer':buf,'variable':var,**st})
    pd.DataFrame(eq_list).to_csv('data/equity_analysis.csv', index=False)
    # Demographic coverage JSON & CSV
    with open('data/coverage_by_demographics.json','w') as f: json.dump(demo_cov, f, indent=2)
    cv_list = []
    for col, cats in demo_cov.items():
        for cat, st in cats.items(): cv_list.append({'demographic_group':col,'category':cat,**st})
    pd.DataFrame(cv_list).to_csv('data/coverage_statistics.csv', index=False)
    # Summary of dynamic weights and overall coverage
    summary = {'dynamic_weights':dyn_wts,'total_tracts':len(census),'total_stations':len(stations),'coverage_percent':float(census['is_served'].mean()*100)}
    with open('data/analysis_summary.json','w') as f: json.dump(summary, f, indent=2)
    
    # Calculate statistics by gap category
    gap_stats = {}
    priority_levels = census['gap_category'].unique()
    for level in priority_levels:
        tracts = census[census['gap_category'] == level]
        gap_stats[level] = {
            'tract_count': int(len(tracts)),
            'population': int(tracts['total_pop'].sum()),
            'area_sq_mi': float(tracts.geometry.area.sum() / (5280 * 5280)),  # Convert sq ft to sq mi
            'pop_density': float(tracts['total_pop'].sum() / (tracts.geometry.area.sum() / (5280 * 5280))),
            'median_income_avg': float(tracts['median_income'].mean()),
            'poverty_rate_avg': float(tracts['poverty_rate'].mean())
        }
    
    # Calculate summary stats for README
    coverage_percent = float(census[census['is_served']]['total_pop'].sum() / census['total_pop'].sum() * 100)

    # Create more detailed README with buffer distances
    readme = f"""
# EV Charging Gap Analysis Results

**Analysis Date:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Overview
This analysis identifies areas in Philadelphia with the greatest need for additional EV charging infrastructure, 
based on physical access to existing chargers and socioeconomic factors.

## Methodology
The gap score combines:
- **Coverage score** ({dyn_wts['coverage']*100:.1f}%): Physical access to existing chargers
- **Socioeconomic scores** ({(1-dyn_wts['coverage'])*100:.1f}%):
  - Income ({dyn_wts['income']*100:.1f}%)
  - Poverty ({dyn_wts['poverty']*100:.1f}%)
  - Population density ({dyn_wts['density']*100:.1f}%)
  - Education ({dyn_wts['education']*100:.1f}%)

## Service Area Buffer Distances
| Mode | Distance | Time Equivalent |
|------|----------|-----------------|
| Walking | 1,320 ft | ~5 min walk |
| Walking | 2,640 ft | ~10 min walk |
| Walking | 3,960 ft | ~15 min walk |
| Driving | 5,280 ft | ~1 mile |
| Driving | 10,560 ft | ~2 miles |
| Driving | 15,840 ft | ~3 miles |

## Priority Categories
- **Low Priority:** gap_score ≤ {THRESHOLDS[0]}
- **Medium Priority:** {THRESHOLDS[0]} < gap_score ≤ {THRESHOLDS[1]}
- **High Priority:** {THRESHOLDS[1]} < gap_score ≤ {THRESHOLDS[2]}
- **Critical Priority:** gap_score > {THRESHOLDS[2]}

## Results Summary
- Total census tracts: {len(census)}
- Total EV stations: {len(stations)}
- Population coverage: {coverage_percent:.1f}%
- Critical priority areas: {gap_stats.get('Critical Priority', {}).get('tract_count', 0)} tracts

## Output Files
- **gap_analysis_fixed.gpkg**: GeoPackage with spatial layers
- **analysis_results.json**: Complete nested analysis results
- **summary_statistics.json**: Key population statistics by category
- **equity_analysis.csv**: Detailed equity analysis results
- **coverage_statistics.csv**: Coverage by demographic group
- **visualizations/**: Charts, maps and summary tables
"""
    with open('data/README.md','w') as f: f.write(readme)

    # Save complete analysis results (nested structure)
    complete_results = {
        'summary': {
            'total_census_tracts': len(census),
            'total_stations': len(stations),
            'total_population': int(census['total_pop'].sum()),
            'study_area_sq_mi': float(census.geometry.area.sum() / (5280 * 5280))
        },
        'coverage': {
            'served_tracts': int(census['is_served'].sum()),
            'served_population': int(census[census['is_served']]['total_pop'].sum()),
            'service_coverage_pct': coverage_percent
        },
        'gap_scores': {
            'mean': float(census['gap_score'].mean()),
            'median': float(census['gap_score'].median()),
            'min': float(census['gap_score'].min()),
            'max': float(census['gap_score'].max()),
            'std': float(census['gap_score'].std())
        },
        'priority_areas': gap_stats,
        'weights': dyn_wts,
        'demographic_coverage': demo_cov,
        'validation': census['validation'] if 'validation' in census.columns else {}
    }
    
    with open('data/analysis_results.json', 'w') as f:
        json.dump(complete_results, f, indent=2, default=str)
    
    # Save summary statistics with population counts by category
    summary_stats = {
        'total_tracts': len(census),
        'total_stations': len(stations),
        'total_population': int(census['total_pop'].sum()),
        'area_sq_mi': float(census.geometry.area.sum() / (5280 * 5280)),
        'population_density': float(census['total_pop'].sum() / (census.geometry.area.sum() / (5280 * 5280))),
        'coverage': {
            'served_population': int(census[census['is_served']]['total_pop'].sum()),
            'coverage_percent': coverage_percent
        },
        'gap_categories': {
            level: {
                'tract_count': gap_stats[level]['tract_count'],
                'population': gap_stats[level]['population']
            } for level in priority_levels
        }
    }
    
    with open('data/summary_statistics.json', 'w') as f:
        json.dump(summary_stats, f, indent=2)


# ----------------------------------------------------------
# Generate visualizations: histogram, equity summary table, bar chart, and interactive map
def create_visualizations(census, equity_res, demo_cov):
    """Generate visualizations: histogram, equity summary table, bar chart, and interactive map."""
    # Create the directory if it doesn't exist
    vis_dir = 'data/visualizations'
    os.makedirs(vis_dir, exist_ok=True)
    
    # Make sure the directory is writable
    if not os.access(vis_dir, os.W_OK):
        print(f"ERROR: Directory {vis_dir} is not writable!")
        return
    
    # 1) Histogram of gap scores with threshold lines and priority labels
    try:
        # Set up the figure
        plt.figure(figsize=(14, 8))
        
        # Create the histogram with gap scores
        n, bins, patches = plt.hist(census['gap_score'], 
                                   bins=30, 
                                   color='steelblue', 
                                   edgecolor='black', 
                                   alpha=0.8)
        
        # Add vertical lines for score thresholds
        for i, threshold in enumerate(THRESHOLDS):
            plt.axvline(x=threshold, color='red', linestyle='--', alpha=0.7, linewidth=2)
            
            # Add label above the line with a white background for better readability
            label_y = max(n) * 0.85  # Position at 85% of max height
            
            # Create a white background for the label
            bbox_props = dict(boxstyle="round,pad=0.3", fc="white", ec="red", alpha=0.8)
            
            # Position the label slightly to the right of the line to avoid overlap
            plt.text(threshold + 0.01, label_y, f"{threshold:.2f}", 
                    color='red', fontweight='bold', ha='left', va='center',
                    bbox=bbox_props)
        
        # Add priority area labels centered in each section
        priorities = ["Low Priority", "Medium Priority", "High Priority", "Critical Priority"]
        section_mids = [0.375, (THRESHOLDS[0]+THRESHOLDS[1])/2, 
                        (THRESHOLDS[1]+THRESHOLDS[2])/2, (THRESHOLDS[2] + 0.8)/2]
        
        for i, (priority, x_pos) in enumerate(zip(priorities, section_mids)):
            plt.text(x_pos, max(n) * 0.95, priority, 
                    ha='center', va='center', fontsize=12, fontweight='bold')
        
        plt.title('Distribution of Gap Scores with Fixed Thresholds', fontsize=14)
        plt.xlabel('Gap Score')
        plt.ylabel('Number of Tracts')
        plt.grid(True, alpha=0.3)
        
        # Save the histogram
        hist_file = f'{vis_dir}/gap_score_distribution.png'
        plt.savefig(hist_file, dpi=200)
        plt.close()  # Make sure to close the plot to avoid displaying it
        
        if os.path.exists(hist_file) and os.path.getsize(hist_file) > 0:
            print(f"Successfully created histogram: {hist_file}")
        else:
            print(f"ERROR: Failed to create histogram or file is empty: {hist_file}")
            
    except Exception as e:
        print(f"ERROR creating histogram: {str(e)}")
    
    # 2) Demographic equity summary CSV
    try:
        if DEBUG_MODE:
            print("Equity results structure:", equity_res.keys())
            print("First buffer sample:", list(equity_res.values())[0] if equity_res else "Empty")

        rows = []
        for buffer_name, metrics in equity_res.items():
            for variable, stats in metrics.items():
                row = {'buffer': buffer_name, 'variable': variable}
                for stat_name, stat_value in stats.items():
                    row[stat_name] = stat_value
                rows.append(row)
        
        if not rows:
            print("WARNING: No equity data available for CSV export!")
            
        eq_df = pd.DataFrame(rows)

        if DEBUG_MODE:
            print(f"Equity DataFrame shape: {eq_df.shape}")
            if not eq_df.empty:
                print(eq_df.head(2))

        # Save to visualizations subfolder
        csv_file = f'{vis_dir}/demographic_equity_summary.csv'
        eq_df.to_csv(csv_file, index=False)
        
        # Check that file was created successfully
        if os.path.exists(csv_file) and os.path.getsize(csv_file) > 0:
            print(f"Successfully created equity CSV: {csv_file}")
        else:
            print(f"ERROR: Failed to create equity CSV or file is empty: {csv_file}")
            
    except Exception as e:
        print(f"ERROR creating equity CSV: {str(e)}")
    
    # 3) Bar chart of coverage by demographic group
    try:
        # Debug the input data
        if DEBUG_MODE:
            print("\nDemographic coverage data:")
            print(f"Keys: {demo_cov.keys()}")
            for col, cats in demo_cov.items():
                print(f"  {col}: {len(cats)} categories")
                for cat, stats in cats.items():
                    print(f"    {cat}: {stats}")
        
        # Prepare data for plotting
        categories = ['density', 'education', 'income']
        levels = ['low', 'medium', 'high']
        
        # Create dataframe with simplified structure
        plot_data = []
        
        for category in categories:
            cat_key = f"{category}_category"
            if cat_key in demo_cov:
                for level in levels:
                    level_key = f"{level}_{category}"
                    if level_key in demo_cov[cat_key]:
                        coverage = demo_cov[cat_key][level_key]['coverage_percent']
                        plot_data.append({
                            'category': category.title(),
                            'level': level.title(),
                            'coverage': coverage
                        })
        
        # Convert to DataFrame
        plot_df = pd.DataFrame(plot_data)
        
        if plot_df.empty:
            print("WARNING: No demographic coverage data available for bar chart!")
            return
            
        if DEBUG_MODE:
            print(f"\nPlot DataFrame shape: {plot_df.shape}")
            print(plot_df.head())
        
        # Create improved bar chart
        plt.figure(figsize=(12, 7))
        
        # Calculate positions for bars
        categories_n = len(categories)
        levels_n = len(levels)
        width = 0.8 / levels_n  # Bar width
        
        # Colors for bars
        colors = ['#FF8C61', '#FFB56B', '#FDD57E']
        
        # Plot bars for each level within each category
        for i, level in enumerate(levels):
            level_data = plot_df[plot_df['level'] == level.title()]
            x_positions = np.arange(categories_n) + (i - levels_n/2 + 0.5) * width
            plt.bar(x_positions, 
                   level_data['coverage'], 
                   width=width, 
                   color=colors[i % len(colors)],
                   label=level.title())
        
        # Set chart title and labels
        plt.title('Service Coverage by Census Tract Characteristic', fontsize=14)
        plt.ylabel('Coverage Percentage')
        plt.ylim(0, 55)  # Set y limit to give space for labels
        
        # Set x-axis ticks - use an empty string to remove the labels
        plt.xticks(np.arange(categories_n), [''] * categories_n)
        
        # Add level labels beneath each category
        for i, category in enumerate(categories):
            for j, level in enumerate(levels):
                x_pos = i + (j - levels_n/2 + 0.5) * width
                plt.text(x_pos, -5, level.title(), 
                        ha='center', fontsize=10)
        
        # Add bracket annotations for categories
        for i, category in enumerate(categories):
            # Draw bracket from first to last bar in category
            first_x = i - width * levels_n/2 + width/2
            last_x = i + width * levels_n/2 - width/2
            mid_x = i
            
            # Bracket height
            y_bracket = -8
            bracket_height = 2
            
            # Draw the horizontal lines
            plt.plot([first_x, last_x], [y_bracket, y_bracket], 'k-', lw=1.5)
            
            # Add the category label
            plt.text(mid_x, y_bracket - bracket_height - 2, category.title(), 
                    ha='center', fontweight='bold', fontsize=12)
        
        plt.grid(True, alpha=0.3, axis='y')
        plt.ylim(bottom=-14)  # Extend bottom margin for labels and brackets
        plt.tight_layout()
        
        # Save and verify file
        bar_file = f'{vis_dir}/demographic_coverage.png'
        plt.savefig(bar_file, dpi=200)
        plt.close()
        
        # Check that file was created successfully
        if os.path.exists(bar_file) and os.path.getsize(bar_file) > 0:
            print(f"Successfully created bar chart: {bar_file}")
        else:
            print(f"ERROR: Failed to create bar chart or file is empty: {bar_file}")
            
    except Exception as e:
        print(f"ERROR creating bar chart: {str(e)}")
    
    # 4) Interactive Folium map of priority areas and stations
    try:
        m = folium.Map(location=[39.9526, -75.1652], zoom_start=11, tiles='CartoDB positron')
        # Priority colors
        colors = {'Low Priority':'#fee5d9','Medium Priority':'#fcae91','High Priority':'#fb6a4a','Critical Priority':'#cb181d'}
        
        # Add tracts
        for _, r in census.to_crs('EPSG:4326').iterrows():
            fc = folium.GeoJson(
                shapely_mapping(r['geometry']), 
                style_function=lambda f,cat=r['gap_category']: {
                    'fillColor': colors.get(cat,'gray'),
                    'color': 'black',
                    'weight': 1,
                    'fillOpacity': 0.7
                },
                # Enhanced tooltip with more tract info
                tooltip=f"Priority: {r['gap_category']}<br>Population: {int(r['total_pop']) if not pd.isna(r['total_pop']) else 'N/A'}<br>Income: ${int(r['median_income']):,} " if not pd.isna(r['median_income']) else "Priority: {r['gap_category']}<br>Population: {int(r['total_pop']) if not pd.isna(r['total_pop']) else 'N/A'}<br>Income: N/A"
            )
            fc.add_to(m)
            
        # Add stations
        for _, s in stations.to_crs('EPSG:4326').iterrows():
            folium.CircleMarker(
                location=[s.geometry.y, s.geometry.x],
                radius=5,
                color='black',
                fill=True,
                fill_color='white',
                fill_opacity=1,
                # Enhanced tooltip with station info
                tooltip=f"Station: {s['name']}<br>Points: {int(s['num_points']) if not pd.isna(s['num_points']) else 'N/A'}<br>Max Power: {s['max_power']:.1f} kW" if not pd.isna(s['max_power']) else f"Station: {s['name']}<br>Points: {int(s['num_points']) if not pd.isna(s['num_points']) else 'N/A'}<br>Max Power: N/A"
            ).add_to(m)

        # Add a legend
        legend_html = '''
        <div style="position: fixed; 
                   bottom: 50px; right: 50px; 
                   border:2px solid grey; z-index:9999; 
                   background-color:white;
                   padding:10px;
                   font-size:14px;
                   ">
        <p><b>Priority Levels</b></p>
        <p><i style="background: #fee5d9; padding:5px;">&nbsp;&nbsp;&nbsp;&nbsp;</i> Low Priority</p>
        <p><i style="background: #fcae91; padding:5px;">&nbsp;&nbsp;&nbsp;&nbsp;</i> Medium Priority</p>
        <p><i style="background: #fb6a4a; padding:5px;">&nbsp;&nbsp;&nbsp;&nbsp;</i> High Priority</p>
        <p><i style="background: #cb181d; padding:5px;">&nbsp;&nbsp;&nbsp;&nbsp;</i> Critical Priority</p>
        <p><i style="background: white; border: 2px solid black; padding:5px;">&nbsp;&nbsp;&nbsp;&nbsp;</i> EV Station</p>
        </div>
        '''
        m.get_root().html.add_child(folium.Element(legend_html))
        
        # Save and verify file
        map_file = f'{vis_dir}/priority_areas_map.html'
        m.save(map_file)
        
        # Check that file was created successfully
        if os.path.exists(map_file) and os.path.getsize(map_file) > 0:
            print(f"Successfully created interactive map: {map_file}")
        else:
            print(f"ERROR: Failed to create interactive map or file is empty: {map_file}")
            
    except Exception as e:
        print(f"ERROR creating interactive map: {str(e)}")


# Filter stations by charging speed and minimum points 
def filter_stations(stations, charging_speed='All', min_points=1):
    """
    Filter charging stations by charging speed category and minimum number of points.
    
    Parameters:
    -----------
    stations : GeoDataFrame
        Station data with num_points and charging_speed columns
    charging_speed : str
        One of: 'All', 'Level 1 (Slow)', 'Level 2 (Medium)', 'DC Fast (Rapid)'
    min_points : int
        Minimum number of charging points (outlets) per station
    
    Returns:
    --------
    GeoDataFrame
        Filtered station dataset
    """
    filtered = stations.copy()
    
    if charging_speed != 'All':
        filtered = filtered[filtered['charging_speed'] == charging_speed]
    
    filtered = filtered[filtered['num_points'] >= min_points]
    
    print(f"Filtered from {len(stations)} to {len(filtered)} stations " +
          f"(speed: {charging_speed}, min points: {min_points})")
    
    return filtered


def snap_points_to_network(points_gdf, network, cache_name=None):
    """
    Snap points to the nearest edge on the network and return a new GeoDataFrame
    with the same attributes but snapped geometries.
    
    This improves network analysis accuracy by ensuring that origins and destinations
    are properly connected to the network.
    
    Parameters:
    -----------
    points_gdf : GeoDataFrame
        Points to snap to the network
    network : networkx.Graph
        Road network 
    cache_name : str, optional
        Name to use for caching the snapped points. If provided, will try to load
        from cache first, and save to cache if not found.
        
    Returns:
    --------
    GeoDataFrame
        New GeoDataFrame with original attributes but snapped geometries
    """
    # Check if we should use cached snapped points
    if cache_name:
        cache_file = os.path.join(CACHE_DIR, f'snapped_{cache_name}.pkl')
        
        # Try to load from cache
        if os.path.exists(cache_file):
            try:
                print(f"Loading snapped points from cache: {cache_name}...")
                with open(cache_file, 'rb') as f:
                    snapped_gdf = pickle.load(f)
                print(f"Loaded {len(snapped_gdf)} snapped points from cache.")
                return snapped_gdf
            except Exception as e:
                print(f"Error loading snapped points cache: {e}")
                # Continue with snapping
    
    print(f"Snapping {len(points_gdf)} points to network...")
    snapped_points = []
    failed_snaps = 0
    
    for idx, row in tqdm(points_gdf.iterrows(), total=len(points_gdf), desc="Snapping points"):
        try:
            # Find the nearest edge
            nearest_edge = ox.distance.nearest_edges(
                network, row.geometry.x, row.geometry.y, return_dist=False)
            
            # Get the edge geometry (might be a LineString or a geometry attribute)
            u, v, _ = nearest_edge  # Unpack edge tuple
            
            if 'geometry' in network.edges[nearest_edge]:
                # If edge has a geometry attribute, use it
                edge_geom = network.edges[nearest_edge]['geometry']
            else:
                # Otherwise, create a line between nodes
                u_x, u_y = network.nodes[u]['x'], network.nodes[u]['y']
                v_x, v_y = network.nodes[v]['x'], network.nodes[v]['y']
                edge_geom = LineString([(u_x, u_y), (v_x, v_y)])
            
            # Snap point to edge (project point onto line)
            projected_point = edge_geom.interpolate(
                edge_geom.project(row.geometry)
            )
            
            # Create new row with snapped geometry
            new_row = row.copy()
            new_row.geometry = projected_point
            snapped_points.append(new_row)
            
        except Exception as e:
            # If snapping fails, keep the original point
            if DEBUG_MODE:
                print(f"Failed to snap point {idx}: {str(e)[:100]}... Using original point.")
            failed_snaps += 1
            snapped_points.append(row)
    
    if failed_snaps > 0:
        print(f"Warning: {failed_snaps} points ({failed_snaps/len(points_gdf)*100:.1f}%) could not be snapped to the network.")
    
    # Create a new GeoDataFrame with the same attributes
    snapped_gdf = gpd.GeoDataFrame(snapped_points, crs=points_gdf.crs)
    
    # Cache the results if cache_name is provided
    if cache_name:
        try:
            cache_file = os.path.join(CACHE_DIR, f'snapped_{cache_name}.pkl')
            with open(cache_file, 'wb') as f:
                pickle.dump(snapped_gdf, f)
            print(f"Saved snapped points to cache: {cache_name}")
        except Exception as e:
            print(f"Failed to save snapped points cache: {e}")
    
    return snapped_gdf


# Main entry point to run the complete pipeline
if __name__ == '__main__':
    # 1) Load data and networks
    census, boundary, stations, validation = load_data()
    
    # Create NetworkX networks
    G_walk, G_drive = create_networks()
    
    print("Using NetworkX for network analysis with point snapping")
    
    # Optional: Filter stations (uncomment and adjust parameters as needed)
    # Possible values for charging_speed: 'All', 'Level 1 (Slow)', 'Level 2 (Medium)', 'DC Fast (Rapid)'
    # stations = filter_stations(stations, charging_speed='All', min_points=1)
    
    # Preprocess: Snap points to network for better connectivity
    print("\n=== PREPROCESSING POINTS FOR NETWORK ANALYSIS ===")
    # Snap stations to the road network
    stations_snapped = snap_points_to_network(stations, G_drive, cache_name='stations')
    print(f"Using {len(stations_snapped)} snapped stations for network analysis")
    
    # Create a GeoDataFrame of census tract centroids
    census_centroids = gpd.GeoDataFrame(
        geometry=[point for point in census.geometry.centroid],
        index=census.index,
        crs=census.crs
    )
    # Snap centroids to the road network
    centroids_snapped = snap_points_to_network(census_centroids, G_drive, cache_name='centroids')
    print(f"Using {len(centroids_snapped)} snapped census tract centroids for network analysis")
    print("Preprocessing complete. Points are now properly connected to the network.")
    
    # 2) Distances & initial coverage
    # Use the snapped geometries for better network connectivity
    origins = {i: row.geometry for i, row in centroids_snapped.iterrows()}
    walk_d, drive_d = calculate_multimodal_distance_batch(
        origins, 
        list(stations_snapped.geometry), 
        G_walk, 
        G_drive
    )
    cov = calculate_coverage_scores(census, walk_d, drive_d, stations)
    census = census.join(cov)
    
    # Flag served tracts
    census['is_served'] = census['coverage_score'] > census['coverage_score'].mean()
    
    # 3) Service areas
    # Use snapped stations for better service area generation
    areas = build_service_areas(stations_snapped, G_walk, G_drive)
    
    # 4) Equity analysis
    equity_res = perform_equity_analysis(census, areas)
    
    # 5) Compute dynamic weights and apply gap scoring
    dyn_wts = calculate_dynamic_weights(equity_res)
    census = apply_gap_scoring(census, dyn_wts)
    
    # 6) Categorize demographics by quantiles
    census['income_category'] = pd.qcut(census['median_income'], 3, labels=['low_income','medium_income','high_income'])
    census['education_category'] = pd.qcut(census['bach_degree_rate'], 3, labels=['low_education','medium_education','high_education'])
    census['density_category'] = pd.qcut(census['pop_density'], 3, labels=['low_density','medium_density','high_density'])
    
    # 7) Coverage by demographic groups
    demo_cov = compute_coverage_by_demographics(census)
    
    # 8) Save outputs and visualizations
    save_pipeline_outputs(census, stations, areas, equity_res, demo_cov, dyn_wts)
    create_visualizations(census, equity_res, demo_cov)

    # Print equity analysis results if in debug mode
    if DEBUG_MODE:
        print("\n=== EQUITY ANALYSIS SUMMARY ===")
        # Calculate average values for served vs unserved areas
        served = census[census['is_served']]
        unserved = census[~census['is_served']]
        
        # Print validation statistics first
        print("\nValidation Statistics:")
        print(f"- Total stations from API: {validation['total_stations_from_api']}")
        print(f"- Stations within boundary: {validation['stations_within_boundary']}")
        print(f"- Excluded stations: {validation['excluded_stations']}")
        print(f"  - Outside boundary: {validation['exclusion_reasons']['outside_boundary']}")
        print(f"  - Missing coordinates: {validation['exclusion_reasons']['missing_coordinates']}")
        
        # Print service area info if available
        print("\nService Areas:")
        if areas:
            for area_name, area_geom in areas.items():
                if not area_geom:
                    print(f"- {area_name}: Empty geometry")
                    continue
                    
                # Count intersecting tracts
                intersecting = sum(1 for _, tract in census.iterrows() if tract.geometry.intersects(area_geom))
                percent = (intersecting / len(census)) * 100
                print(f"- {area_name}: {intersecting} tracts ({percent:.1f}%)")
        else:
            print("No service areas were created.")
        
        # Print metric comparisons (original code)
        print("\nServed vs. Unserved Areas:")
        metrics = ['median_income', 'poverty_rate', 'pop_density', 'bach_degree_rate']
        print(f"{'Metric':<20} {'Served':>12} {'Unserved':>12} {'Percent Diff':>12} {'Significant':>10}")
        print("-" * 70)
        
        for var in metrics:
            served_mean = served[var].mean()
            unserved_mean = unserved[var].mean()
            pct_diff = ((served_mean - unserved_mean) / unserved_mean) * 100 if unserved_mean != 0 else 0
            t_stat, p_val = stats.ttest_ind(served[var].dropna(), unserved[var].dropna())
            
            print(f"{var:<20} {served_mean:>12.1f} {unserved_mean:>12.1f} {pct_diff:>+12.1f}% {'Yes' if p_val < 0.05 else 'No':>10}")

    # Print dynamic weights if in debug mode
    if DEBUG_MODE:
        print("\n=== DYNAMIC WEIGHTS ===")
        for k, v in dyn_wts.items():
            print(f"{k}: {v:.4f} ({v*100:.1f}%)")

    # Add validation to pipeline outputs
    census['validation'] = validation
    print('Complete gap analysis pipeline.') 

Working directory: c:\Users\rache\Documents\musa 550 FINAL\quarto_musa_550\analysis
OSMnx cache folder: data\cache\osmnx
Using NetworkX with point snapping for network analysis.
Using NetworkX for network analysis with point snapping

=== PREPROCESSING POINTS FOR NETWORK ANALYSIS ===
Loading snapped points from cache: stations...
Loaded 130 snapped points from cache.
Using 130 snapped stations for network analysis
Loading snapped points from cache: centroids...
Loaded 385 snapped points from cache.
Using 385 snapped census tract centroids for network analysis
Preprocessing complete. Points are now properly connected to the network.
Loading distances from cache...
Loaded 50050 walking and 50050 driving distances from cache.
Loading service areas from cache...
Loaded 5 service areas from cache
Loading equity analysis from cache...
Loaded equity analysis for 5 service areas from cache
Original weights: {'coverage': 0.3, 'income': 0.2, 'poverty': 0.30000000000000004, 'density': 0.25, 'educ

C:\Users\rache\AppData\Local\Temp\ipykernel_23112\409186455.py:1010: RuntimeWarning: invalid value encountered in scalar divide
  'pop_density': float(tracts['total_pop'].sum() / (tracts.geometry.area.sum() / (5280 * 5280))),


Successfully created histogram: data/visualizations/gap_score_distribution.png
Equity results structure: dict_keys(['walk_400m', 'walk_800m', 'drive_1000m', 'drive_3000m', 'drive_8000m'])
First buffer sample: {'median_income': {'effect_size': -0.12465946575488303, 'ci_lower': -6684.376767083484, 'ci_upper': 6684.127448151974, 'p_value': 0.24716139952343424}, 'poverty_rate': {'effect_size': 0.6365812002143018, 'ci_lower': -2.2308921974977856, 'ci_upper': 3.504054597926389, 'p_value': 6.057475773524907e-09}, 'pop_density': {'effect_size': 0.7496382501362957, 'ci_lower': -2404.4217684494174, 'ci_upper': 2405.92104494969, 'p_value': 1.105167556086776e-11}, 'bach_degree_rate': {'effect_size': 0.30932633445014873, 'ci_lower': -1.821517594859319, 'ci_upper': 2.4401702637596165, 'p_value': 0.00405668195382175}}
Equity DataFrame shape: (20, 6)
      buffer       variable  effect_size     ci_lower     ci_upper  \
0  walk_400m  median_income    -0.124659 -6684.376767  6684.127448   
1  walk_400m 

# Methods

## Data & Pre‑processing
- **Census Data**: ACS data loaded from GeoPackage
- **Filters**: Tracts with population > 0 and density ≥ 1000 people/mi²
- **Filtered Tracts**: 385 census tracts remained after filtering, with 23 tracts excluded from the original 408 Philadelphia tracts
- **Boundary**: Philadelphia city limits from GeoJSON
- **Projection**: WGS84 (4326) initially, then State Plane (2272) for analysis

## EV Station Retrieval
- **API**: OpenChargeMap v3 with 10km search radius
- **Station Data**: Extracts coordinates, connection points, max power (kW)
- **Station Filtering**: Of 150 stations retrieved from the API, 20 were excluded for being outside the city boundary, leaving 130 stations for analysis
- **Station Classification**:
  - Level 1 (Slow): < 7 kW
  - Level 2 (Medium): 7-50 kW
  - DC Fast (Rapid): > 50 kW
- **Validation**: Tracks excluded stations and reasons (outside boundary, missing coordinates)

## Network Analysis
- **Networks**: OSMnx-generated walking and driving networks
- **Point Snapping**: Stations and census tract centroids are snapped to the nearest edge on the network for accurate routing
- **Distance Calculation**: NetworkX-based shortest path routing with straight-line distance fallback
- **Distance Filter**: 6km threshold to eliminate unnecessary calculations for distant pairs
- **Caching**: Extensive caching of networks, snapped points, distances, service areas, and equity analysis for performance
- **Service Areas**:
  - Walking: 1,320ft (5min), 2,640ft (10min), 3,960ft (15min)
  - Driving: 5,280ft (1mi), 10,560ft (2mi), 15,840ft (3mi)

## Methodology: Computing the EV Charging "Gap Score"

The **gap score** is a single composite metric that ranks each Philadelphia census tract by its need for additional EV charging infrastructure. It blends:

1. A **coverage** metric (physical access to existing chargers)  
2. Four **socioeconomic vulnerability** metrics (income, poverty, density, education)

Together, these capture both *where* chargers are missing **and** *who* is most harmed by those gaps.

### 1. Coverage Score (24%)

#### What it measures  
Per-tract service level based on walking and driving proximity, station quality, and population density.

#### Calculation steps  
1. **Compute network distances**  
   - Shortest-path walking (NetworkX with point snapping) from each tract centroid to all chargers.
   - Shortest-path driving likewise.
   - Straight-line distance with detour factor (1.3× for walking, 1.5× for driving) as fallback when routing fails.

2. **Apply proximity bands**  
   - **Walking**:  
     - Within 1,320 ft (~5 min): 0.7 score
     - Within 2,640 ft (~10 min): 0.5 score
     - Within 3,960 ft (~15 min): 0.3 score
     - Beyond: 1.0 score (worst)
   
   - **Driving**:  
     - Within 5,280 ft (~1 mile): 0.7 score
     - Within 10,560 ft (~2 miles): 0.5 score
     - Within 15,840 ft (~3 miles): 0.3 score
     - Beyond: 1.0 score (worst)

3. **Station quality adjustment**  
   - Calculate quality index for each station:
     $$\text{quality\_index} = \text{num\_points} \times \sqrt{\frac{\text{max\_power}}{50}}$$
   - Normalize to 0.5-1.5 range: 
     $$\text{quality\_factor} = 0.5 + \frac{\text{quality\_index} - \min(\text{quality\_index})}{\max(\text{quality\_index}) - \min(\text{quality\_index})}$$
   - Multiply coverage by quality factor

4. **Combine modes**  
   $$\text{coverage\_score}_i = 0.4 \times \text{walk\_weight}_i + 0.6 \times \text{drive\_weight}_i$$

5. **Density adjustment**  
   - Scale down by up to 30% for high-density tracts:
   $$\text{coverage\_score}_i \times= (1 - 0.3 \times \frac{\text{pop\_density}_i}{\max(\text{pop\_density})})$$

6. **Label "served" vs. "unserved"**  
   - **Served** if coverage_score > mean(coverage_score)  
   - **Unserved** otherwise  

### 2. Socioeconomic Component Scores (Combined 76%)

We compute four additional 0–1 scores that capture relative vulnerability:

| Component      | Weight | Rationale                                                         |
|:---------------|:------:|:------------------------------------------------------------------|
| **Income**     | 16%    | Lower-income areas may have fewer EV adopters and fewer resources to retrofit homes. |
| **Poverty**    | 24%    | High poverty correlates with lower EV uptake and higher energy burden. |
| **Density**    | 20%    | More people in a tract → higher absolute charging demand.        |
| **Education**  | 16%    | Education level often correlates with technology adoption rates.  |

#### How we compute each score  
1. Pull ACS census data for each tract.  
2. For each metric (e.g. median income), scale the raw value so that the **most vulnerable** tract gets 1.0 and the **least vulnerable** gets 0.0.  
   - e.g. `income_score_i = 1 - (income_i − min)/ (max − min)`  
3. Clip to [0, 1] range.

### 3. Combining into the Gap Score

For each tract $i$, we use a simple linear combination of the normalized components:

$$\begin{aligned}
\text{gap\_score}_i 
&= w_{\text{cov}}(1 - \text{coverage}_i) \\
&\quad + w_{\text{inc}}\text{income\_score}_i 
+ w_{\text{pov}}\text{poverty\_score}_i \\
&\quad + w_{\text{den}}\text{density\_score}_i 
+ w_{\text{edu}}\text{education\_score}_i
\end{aligned}$$

This linear model ensures transparency and interpretability in how each factor contributes to the final score.

#### Component directions:
- **Coverage**: Inverted (1 - score) so higher values indicate worse coverage
- **Income**: Inverted so lower incomes result in higher scores
- **Poverty**: Higher poverty rates result in higher scores
- **Density**: Higher population density results in higher scores
- **Education**: Inverted so lower education levels result in higher scores

#### Dynamic weights
Weights are calculated based on observed disparities:

1. We start with base weights that sum to 1.0:
   - $w_{\text{cov}} = 0.30$  
   - $w_{\text{inc}} = 0.20$  
   - $w_{\text{pov}} = 0.20$  
   - $w_{\text{den}} = 0.15$  
   - $w_{\text{edu}} = 0.15$

2. We calculate effect sizes (Cohen's d) for each socioeconomic variable between served and unserved areas.

3. For metrics with statistically significant disparities (p < 0.05), we adjust weights:
   - Small effect (0.2-0.5): +5% weight
   - Medium effect (0.5-0.8): +10% weight
   - Large effect (>0.8): +15% weight

4. We then re-normalize all weights to sum to 1.0.

5. In our implementation, this resulted in the following weights:
   - $w_{\text{cov}} = 0.24$  
   - $w_{\text{inc}} = 0.16$  
   - $w_{\text{pov}} = 0.24$  
   - $w_{\text{den}} = 0.20$  
   - $w_{\text{edu}} = 0.16$

### 4. Thresholding & Priority Tiers

To turn a continuous score into actionable categories, we use fixed thresholds:

- **Low Priority**: gap_score ≤ 0.45
- **Medium Priority**: 0.45 < gap_score ≤ 0.55
- **High Priority**: 0.55 < gap_score ≤ 0.65
- **Critical Priority**: gap_score > 0.65

These thresholds create a balanced distribution of priority levels, with approximately 30% Low, 28% Medium, 29% High, and 13% Critical priority tracts.

### 5. Validation & Outputs

The analysis pipeline tracks:
- Excluded stations and reasons for exclusion
- Demographic coverage across income, education, and density levels
- Statistical significance of socioeconomic disparities
- Component contributions to the final scores
- Distribution of priority categories with tract counts and percentages
---

**Footnote:**  
Tracts with large **group‑quarters** populations (e.g. university dorms) may show artificially high poverty. We document this caveat but retain them in the analysis to avoid geographic bias.

---